In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Define the directory path
data_dir = r"C:\Users\manoj\Downloads\Chicken-fecal-images\Chicken-fecal-images"

# Data Augmentation and Rescaling
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

# Training Data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),    # Resizing images to 224x224
    batch_size=32,
    class_mode='binary',       # Since it's binary classification
    subset='training'          # Use the training subset
)

# Validation Data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'        # Use the validation subset
)


# Data Augmentation and Rescaling
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

# Training Data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),    # Resizing images to 224x224
    batch_size=32,
    class_mode='binary',       # Since it's binary classification
    subset='training'          # Use the training subset
)

# Validation Data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'        # Use the validation subset
)


Found 312 images belonging to 2 classes.
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Found 78 images belonging to 2 classes.


In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the ResNet50 pre-trained model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 321s 3us/step


In [7]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Adjust epochs as needed
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10


C:\Users\manoj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.5253 - loss: 3.3589 - val_accuracy: 0.5156 - val_loss: 1.0223
Epoch 2/10
1/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.4062 - loss: 1.1940

C:\Users\manoj\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - accuracy: 0.4062 - loss: 1.1940 - val_accuracy: 0.5714 - val_loss: 0.6736
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5312 - loss: 0.8320 - val_accuracy: 0.5000 - val_loss: 0.7475
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.5938 - loss: 0.6248 - val_accuracy: 0.5000 - val_loss: 0.6325
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5841 - loss: 0.6049 - val_accuracy: 0.6875 - val_loss: 0.5598
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.7083 - loss: 0.5957 - val_accuracy: 0.3571 - val_loss: 1.0015
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.5968 - loss: 0.7096 - val_accuracy: 0.5469 - val_loss: 0.8201
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.5938 - loss: 0.7260 - val_accuracy: 0.3571 - val_loss: 0.9307
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.6195 - loss: 0.6337 - val_accuracy: 0.8594 - val_loss: 0.4708
Epoch 10/10
9/

In [8]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.7454 - loss: 0.4942
Test accuracy: 0.7564102411270142


In [9]:
# After training the model, save it as 'model.h5'
model.save('cocidoisis_model.h5')
